#### See the [poster](https://github.com/vtomole/SymboliQ/blob/main/examples/poster.pdf) for a description of this code.

In [50]:
from sympy.physics.quantum import qapply
from sympy.physics.quantum.gate import XGate
from sympy.physics.quantum.qubit import Qubit
import symboliq

In [51]:
state = Qubit(0)

XGate(0) * state

X(0)*|0>

In [52]:
qapply(XGate(0) * state)

|1>

In [53]:
XGate(0)._apply_operator_Qubit(state)

|1>

In [54]:
def apply_operator_Qubit(operator, qubits):
    targets = operator.targets
    
    # returns Matrix([[0, 1], [1, 0]]) for an X gate
    target_matrix = operator.get_target_matrix() 

    # Find which column of the target matrix this applies to.
    column_index = 0
    n = 1
    for target in targets:
        column_index += n * qubits[target]
        n = n << 1
        column = target_matrix[:, int(column_index)]

    # Now apply each column element to the qubit.
    result = 0
    for index in range(column.rows):
        new_qubit = qubits.__class__(*qubits.args)
        # Flip the bits that need to be flipped.
        for bit, target in enumerate(targets):
            if new_qubit[target] != (index >> bit) & 1:
                new_qubit = new_qubit.flip(target)
        # The value in that row and column times the 
        # flipped-bit qubit is the result for that part.
        result += column[index] * new_qubit
    return result

In [55]:
apply_operator_Qubit(XGate(0), state)

|1>

In [56]:
symboliq.qapply(XGate(0) * state)

|1>

In [57]:
print(symboliq.get_simp_steps(XGate(0) * state))

(0) X(0)*|0>
(1) |0><1|*|0> + |1><0|*|0>
(2) |1>

